In [1]:
import ee
import pandas as pd
from pathlib import Path
from datetime import datetime, timezone
import os
import re

from google.auth.transport.requests import AuthorizedSession

ee.Authenticate()  #  or !earthengine authenticate --auth_mode=gcloud
session = AuthorizedSession(ee.data.get_persistent_credentials())

## Create empty image collections as target
Create empty image collection with earthengine CLI
Purge contents if delete or clean repopulation needed
List COGs in GCS folder

```
cd /data/gis/raster_base/Alaska/AKVegMap/disturbance

conda activate ee

earthengine create collection projects/akveg-map/assets/disturbance/nbac_annual_v20240530

for i in `earthengine ls projects/akveg-map/assets/disturbance/nbac_annual_v20240530`; do earthengine rm $i; done

gsutil ls gs://akveg-data/disturbance/nbac/v20240530/*.tif > nbac_annual_v20240530_on_gcs.txt
```

In [2]:
import json
# from urllib.parse import urlparse
import os
from pprint import pprint

# Earth Engine enabled Cloud Project.
project_folder = 'akveg-map'
collection = 'disturbance/nbac_annual_v20240530'
cogs = pd.read_csv('/data/gis/raster_base/Alaska/AKVegMap/disturbance/nbac_annual_v20240530_on_gcs.txt', header=None,names=['tif'])


In [9]:
# TODO Fix date extraction from filename to work for different filename formats
# Parts are zero-based
def load_gcs_cogs_to_collection(cogs, project_folder, collection, year_part, version_part):
    # Request body as a dictionary.
    for cog in cogs['tif']:
      fileOnly = os.path.split(cog)[1]
      # print(fileOnly)

      cogName = fileOnly[:-4]
      print(cogName)
      
      # parts = fileOnly.split('_')
      # Split on both underscore and period
      parts = re.split(r'[_\.]', fileOnly)
      print(parts)
      
      year = f"{parts[year_part]}"
      year = int(year)
      print(year)

      version = f"{parts[version_part]}"
      print(version)

      start_dt = datetime(year, 1, 1)
      start_dt = f"{start_dt:%Y-%m-%dT%H:%M:%SZ}"  # Format string for ISO 8601 + Z
      end_dt = datetime(year + 1, 1, 1)
      end_dt = f"{end_dt:%Y-%m-%dT%H:%M:%SZ}"  # Format string for ISO 8601 + Z
      print(start_dt, end_dt)
        
      request = {
        'type': 'IMAGE',
        'gcs_location': {
          'uris': cog
        },
        'properties': {
            'version': version,
            #   'source': 'https://code.earthengine.google.com/067b10ee56537817756a3177a9138aee',
            # 'yyyymmdd_hhmmss': yyyymmdd_hhmmss
        },
        'startTime': start_dt,#'2023-01-01T00:00:00.000000000Z',
        'endTime': end_dt,#'2024-01-01T00:00:00.000000000Z',
      }

      # pprint(json.dumps(request))

      # A folder (or ImageCollection) name and the new asset name.
      asset_id = collection+'/'+cogName
      # print(project_folder)
      # print(asset_id)
        
      url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}'
      print(url)

      response = session.post(
        url = url.format(project_folder, asset_id),
        data = json.dumps(request)
      )

      # pprint(json.loads(response.content))
    print('done')


## Run it

In [10]:
load_gcs_cogs_to_collection(cogs, project_folder, collection, 4, 3)


nbac_30m_3338_v20240530_1972
['nbac', '30m', '3338', 'v20240530', '1972', 'tif']
1972
v20240530
1972-01-01T00:00:00Z 1973-01-01T00:00:00Z
https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}
nbac_30m_3338_v20240530_1973
['nbac', '30m', '3338', 'v20240530', '1973', 'tif']
1973
v20240530
1973-01-01T00:00:00Z 1974-01-01T00:00:00Z
https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}
nbac_30m_3338_v20240530_1974
['nbac', '30m', '3338', 'v20240530', '1974', 'tif']
1974
v20240530
1974-01-01T00:00:00Z 1975-01-01T00:00:00Z
https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}
nbac_30m_3338_v20240530_1975
['nbac', '30m', '3338', 'v20240530', '1975', 'tif']
1975
v20240530
1975-01-01T00:00:00Z 1976-01-01T00:00:00Z
https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}
nbac_30m_3338_v20240530_1976
['nbac', '30m', '3338', 'v20240530', '1976', 'tif']
1976
v20240530
1976-01-01T00:00:00Z 1977-01-01T00:00:00Z
https://earthengin